## Clone Repo

In [ ]:
!cd /root/VoiceRobot
!rm -rf sample_data ChatTTS
!git clone https://github.com/2noise/ChatTTS.git

In [ ]:
!cd /root/VoiceRobot

## Install Requirements

In [ ]:
!pip install -r /content/ChatTTS/requirements.txt
!ldconfig /usr/lib64-nvidia

## Import Packages

In [1]:
import torch

torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision("high")

from ChatTTS import ChatTTS
from ChatTTS.tools.logger import get_logger
from ChatTTS.tools.normalizer import normalizer_en_nemo_text, normalizer_zh_tn
from IPython.display import Audio

/root/.conda/envs/asrrobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Models

In [2]:
logger = get_logger("ChatTTS", format_root=True)
chat = ChatTTS.Chat(logger)

# try to load normalizer
try:
    chat.normalizer.register("en", normalizer_en_nemo_text())
except ValueError as e:
    logger.error(e)
except:
    logger.warning("Package nemo_text_processing not found!")
    logger.warning(
        "Run: conda install -c conda-forge pynini=2.1.5 && pip install nemo_text_processing",
    )
try:
    chat.normalizer.register("zh", normalizer_zh_tn())
except ValueError as e:
    logger.error(e)
except:
    logger.warning("Package WeTextProcessing not found!")
    logger.warning(
        "Run: conda install -c conda-forge pynini=2.1.5 && pip install WeTextProcessing",
    )

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.


2024-09-09 16:16:07,303 WETEXT INFO found existing fst: /root/.conda/envs/asrrobot/lib/python3.10/site-packages/tn/zh_tn_tagger.fst
2024-09-09 16:16:07,304 WETEXT INFO                     /root/.conda/envs/asrrobot/lib/python3.10/site-packages/tn/zh_tn_verbalizer.fst
2024-09-09 16:16:07,305 WETEXT INFO skip building fst for zh_normalizer ...


### Here are three choices for loading models,

#### 1. Load models from Hugging Face (recommend)

In [ ]:
# use force_redownload=True if the weights have been updated.
chat.load(source="huggingface")

#### 2. Load models from local directories 'asset' and 'config'

In [3]:
chat.load()
# chat.load(source='local') same as above

[+0800 20240909 16:16:28] [INFO] ChatTTS | dl | checking assets...
[+0800 20240909 16:16:29] [INFO] ChatTTS | dl | all assets are already latest.
[+0800 20240909 16:16:29] [INFO] ChatTTS | core | use device cuda:0
/root/.conda/envs/asrrobot/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
[+0800 20240909 16:16:31] [INFO] ChatTTS | core | vocos loaded.
[+0800 20240909 16:16:31] [INFO] ChatTTS | core | dvae loaded.
[+0800 20240909 16:16:32] [INFO] ChatTTS | core | embed loaded.
[+0800 20240909 16:16:32] [INFO] ChatTTS | core | gpt loaded.
[+0800 20240909 16:16:32] [INFO] ChatTTS | core | speaker loaded.
[+0800 20240909 16:16:33] [INFO] ChatTTS | core | decoder l

True

#### 3. Load models from a custom path

In [ ]:
# write the model path into custom_path
chat.load(source="custom", custom_path="YOUR CUSTOM PATH")

### You can also unload models to save the memory

In [ ]:
chat.unload()

## Inference

### Batch infer

In [4]:
texts =  [
    "我觉得像我们这些写程序的人，他，我觉得多多少少可能会对开源有一种情怀在吧我觉得开源是一个很好的形式。现在其实最先进的技术掌握在一些公司的手里的话，就他们并不会轻易的开放给所有的人用。"
] 

wavs = chat.infer(texts)

text:   0%|          | 0/384(max) [00:00, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
text:  28%|██▊       | 107/384(max) [00:01, 65.54it/s]
code:  42%|████▏     | 861/2048(max) [00:10, 79.19it/s]


In [5]:
Audio(wavs[0], rate=24_000, autoplay=True)

In [7]:
from scipy.io.wavfile import write

# 假设 wavs[0] 是一个 numpy 数组，包含音频数据
# 24_000 是采样率
write('output.wav', 24000, wavs[0])

In [ ]:
texts = [
    "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with.",
] * 3 + [
    "我觉得像我们这些写程序的人，他，我觉得多多少少可能会对开源有一种情怀在吧我觉得开源是一个很好的形式。现在其实最先进的技术掌握在一些公司的手里的话，就他们并不会轻易的开放给所有的人用。"
] * 3

wavs = chat.infer(texts)

In [ ]:
Audio(wavs[0], rate=24_000, autoplay=True)

In [ ]:
Audio(wavs[3], rate=24_000, autoplay=True)

### Custom params

In [47]:
speakvector_male707 = '4.331780433654785,1.2567427158355713,-7.442240715026855,-4.060439109802246,0.2636751234531402,6.47605562210083,-9.826565742492676,-1.230890154838562,1.2207282781600952,12.844109535217283,-0.2050678133964538,-10.350702285766602,-4.117189884185791,-0.4594649076461792,-2.5871686935424805,-1.1078705787658691,0.547939121723175,-4.015372276306152,-0.973813533782959,-0.7876948118209839,3.076204299926758,4.452235221862793,-0.3988082408905029,-27.453033447265625,-1.851612329483032,-2.166126251220703,-11.42893409729004,9.557198524475098,0.817314863204956,1.8018275499343872,6.459125518798828,-1.5202888250350952,1.68973708152771,-1.278842568397522,0.4236142039299011,-4.618363380432129,-1.105473518371582,2.576324224472046,-1.8023244142532349,1.0216302871704102,1.903223752975464,1.1027157306671145,-1.1188828945159912,1.051018238067627,11.13942527770996,-1.6640262603759766,-2.817953586578369,-4.494587421417236,5.455719470977783,2.024247646331787,-1.276464581489563,0.6893980503082275,-0.370679497718811,4.417150974273682,0.1858023852109909,-0.8312020301818848,-3.931576728820801,2.434983730316162,-5.823186874389648,4.628828048706055,1.3608169555664062,-0.8101356625556946,6.16969633102417,-2.079301357269287,-3.748968362808228,1.0068827867507937,-2.79156494140625,1.4783910512924194,-1.2124011516571045,-0.4918941259384155,4.568844795227051,1.1050405502319336,0.1870990097522735,1.8652009963989256,14.839118003845217,2.869025707244873,2.629971027374268,-0.3622491359710693,-1.930704116821289,6.826616287231445,2.476871967315674,-7.11643123626709,-0.8220564126968384,-0.0730593204498291,2.597143650054932,0.2675676941871643,-2.8345091342926025,2.126636505126953,2.741934061050415,3.024877548217773,13.229873657226562,-1.2386211156845093,1.208616018295288,-3.040659189224243,7.660648822784424,-15.52113151550293,-0.0550375282764434,2.0630195140838623,0.4617520868778229,-0.3007514476776123,1.2226184606552124,0.2843161225318908,-5.037950038909912,0.8102582097053528,0.4978199005126953,3.092647790908813,4.077357292175293,0.7435593605041504,3.5849087238311768,2.431905746459961,4.897055625915527,-23.829235076904297,23.539226531982425,3.064973831176758,2.14164400100708,-3.558781147003174,0.1719150096178054,0.3049731850624084,-1.231790542602539,3.816373825073242,0.965104341506958,1.6699926853179932,2.760582208633423,9.51341438293457,5.46431827545166,-1.771198272705078,2.910351514816284,-2.702591180801392,-2.2025744915008545,2.0593857765197754,2.0943498611450195,0.623497486114502,1.284337878227234,-3.9917168617248535,10.73388671875,-2.3717565536499023,-7.2998046875,-0.9129738211631776,13.321979522705078,1.4114627838134766,-0.7865017056465149,-0.8245707750320435,2.025641202926636,2.6565017700195312,1.8285822868347168,-3.796666145324707,-4.721895217895508,3.477649688720703,1.215101957321167,-0.3595702648162842,1.2293500900268557,1.7526493072509766,-0.996840000152588,9.313739776611328,5.831663608551025,-1.9957534074783323,22.749755859375,-1.7227156162261963,-2.357852935791016,-2.7152202129364014,5.103940963745117,-2.607548952102661,2.4087071418762207,-0.5054129362106323,-1.5235614776611328,-2.5452513694763184,-4.004184246063232,-0.9886434078216552,4.288312911987305,-6.633824348449707,-0.3395283222198486,1.2998417615890503,-0.1695831716060638,0.0490310192108154,-0.1654029935598373,2.243971109390259,4.46951150894165,-2.1687183380126958,0.5093129277229309,4.628344058990479,0.5954830646514893,-1.6252422332763672,0.2861648797988891,-0.5375845432281494,-0.1533312797546386,1.1176851987838743,7.338626861572266,-1.441317081451416,6.688563823699951,5.106064796447754,0.1363435685634613,1.5385677814483645,8.934282302856445,-4.181968688964844,-4.1973419189453125,-0.0312423706054687,0.2837728261947632,-2.492027997970581,-2.929988384246826,-0.3487789332866668,-3.5928354263305664,-1.8356043100357056,3.1445319652557373,-4.492799758911133,-3.4750146865844727,4.239563941955566,-4.505738258361816,3.780493259429932,-4.965705871582031,-3.761109590530396,2.959885835647583,-1.1263474225997925,1.1651065349578855,8.681085586547852,-0.7382983565330505,0.8381307721138,-0.046876735985279,-13.178290367126465,2.3078787326812744,15.269976615905762,1.0168615579605105,-4.875334739685059,1.666407346725464,6.41486120223999,3.736551284790039,-7.740783214569092,-4.973683834075928,1.9807897806167605,-4.657805919647217,1.019670009613037,-1.2295678853988647,3.859128952026367,-4.710142612457275,-5.482114791870117,0.5590587854385376,1.4955021142959597,2.782223701477051,6.9006781578063965,-1.3113372325897217,0.6100229024887085,-3.761909484863281,1.066452145576477,-8.190978050231934,-2.237203598022461,4.6029953956604,-2.1720430850982666,-7.660277843475342,-1.4496493339538574,16.446102142333984,0.7247344255447388,1.2804083824157717,1.8241750001907349,-4.523299217224121,5.238900184631348,12.193941116333008,-0.7077841758728027,3.485487699508667,7.582026481628418,1.643390655517578,-0.2292030453681945,-0.101946473121643,3.3573944568634038,-3.6913270950317374,4.976407051086426,1.6830538511276243,-2.143282175064087,-5.126486301422119,11.440181732177734,-1.2369911670684814,-2.767651081085205,-2.201055765151977,-8.504733085632324,-5.898917675018311,-10.045293807983398,-1.0827782154083252,6.83915901184082,6.89871072769165,-0.3321967720985412,-2.9019229412078857,1.6435561180114746,-1.694788098335266,-0.4377144873142242,-6.624029636383057,6.235098361968994,1.412814736366272,0.6707792282104492,-3.256554126739502,-0.8604064583778381,-5.489842414855957,0.0231299400329589,0.499051570892334,0.1944193840026855,1.6822526454925537,0.2345150709152221,5.447916030883789,-1.6355466842651367,-3.1519522666931152,5.823206901550293,0.4956316947937011,-0.3170292675495147,2.256967544555664,-1.2665972709655762,0.9818007946014404,-2.0583856105804443,-1.0652509927749634,-3.5482070446014404,6.28336238861084,-1.948540449142456,-1.2130966186523438,-3.27761459350586,-4.305275917053223,-0.75895094871521,5.220755100250244,-0.9718878269195556,1.3941848278045654,1.345356822013855,2.9940731525421143,-2.564177989959717,-2.165724277496338,0.4131564497947693,0.6076871156692505,-2.1010656356811523,5.690085411071777,1.2997376918792725,2.4837985038757324,17.42943572998047,-0.0607858896255493,3.7982470989227295,-2.6950526237487797,1.1092236042022705,1.9588348865509035,-4.813918113708496,-5.1000285148620605,-1.9239716529846191,1.285209894180298,11.336247444152832,16.74537467956543,-6.249295711517334,1.7900465726852417,0.8173068761825562,1.4458156824111938,-4.040390968322754,-0.3214460611343384,-4.202798366546631,-0.1252444386482238,1.2689690589904783,-1.0170836448669434,0.4441559314727783,-1.5169262886047363,7.210160732269287,-5.794026374816895,-3.521674156188965,-0.2221859693527221,1.3363633155822754,2.4509878158569336,-2.532780170440674,4.687276363372803,3.042032480239868,3.101653575897217,1.9993184804916384,-1.6297863721847534,-4.698965072631836,-2.979632377624512,2.748786687850952,2.4385433197021484,-13.23155403137207,3.878911256790161,-11.231701850891112,2.0696732997894287,6.479875564575195,1.2949869632720947,-1.6116163730621338,-1.1285722255706787,1.8516554832458496,0.0243409052491188,0.8751246333122253,6.33336353302002,-1.7105152606964111,-2.042419910430908,-0.4489282667636871,-2.166989803314209,-3.007753133773804,3.3629252910614014,-0.1081659793853759,4.28210973739624,13.734716415405272,1.8319785594940183,9.679285049438477,-5.421370506286621,1.594630241394043,2.1615004539489746,0.4743689000606537,1.1368988752365112,-0.3425025343894958,4.719234943389893,0.4485016465187073,2.3406667709350586,-0.6850001215934753,-5.637679100036621,-2.655301570892334,-1.0760642290115356,-1.965854287147522,2.222452163696289,-0.3321196734905243,15.27625846862793,0.8773690462112427,-8.209456443786621,3.5265111923217773,-9.510793685913086,9.512971878051758,0.497325599193573,1.609528422355652,-6.558365821838379,1.675873637199402,-4.92982816696167,1.6079881191253662,-5.111255645751953,10.34829044342041,0.9818158149719238,2.162561416625977,-1.205601692199707,4.776269435882568,1.1388790607452393,-4.4050469398498535,2.0102381706237797,3.378005027770996,0.6904934644699097,15.387813568115234,0.2310463786125183,-1.694166898727417,-4.081916809082031,-1.378847599029541,-2.9048821926116943,-0.9596980810165404,-8.314167022705078,-3.827182531356812,-0.2647929191589355,4.800992488861084,2.6543397903442383,-2.295707941055298,-5.013652801513672,-0.0181173682212829,-0.3107200860977173,1.894003629684448,-6.645004749298096,1.116288661956787,-10.010979652404783,0.585698127746582,-2.243438482284546,4.3741536140441895,-0.6649069786071777,-5.007852554321289,-4.588269710540772,2.014291524887085,3.352785348892212,5.398392677307129,-4.463080883026123,1.5108556747436523,-8.35037612915039,2.0347461700439453,0.4230232238769531,0.1440888792276382,-0.2807742953300476,11.251262664794922,1.3918944597244265,0.8442867994308472,5.3689470291137695,-0.6420419216156006,3.304217338562012,5.884127616882324,-5.0096635818481445,0.0812997817993164,-1.2157037258148191,5.393387794494629,0.0712873935699462,-0.5261030197143555,1.2015929222106934,1.4271007776260376,-1.1711010932922363,-0.2417481541633606,0.3207602500915527,0.8903464078903198,-4.995526313781738,6.994955062866211,-1.1765813827514648,-2.274653434753418,-3.811958551406861,-1.080563187599182,2.0609171390533447,1.7023159265518188,0.8378866910934448,-4.151884078979492,-10.763723373413086,14.559287071228027,-11.05797290802002,5.126373291015625,1.9588568210601809,3.436093330383301,2.865067720413208,3.0218162536621094,-1.342160940170288,1.4344124794006348,6.9466471672058105,-4.802103519439697,4.108699798583984,-14.220574378967283,1.603285312652588,12.293046951293944,5.286745548248291,1.7291988134384155,-0.6853454113006592,-2.1098082065582275,-3.4169578552246094,0.2984036207199096,-1.5008213520050049,0.2816269397735595,-1.5987228155136108,0.1553647518157959,-2.5563018321990967,-3.037681818008423,2.0433349609375,4.465623378753662,-3.740667343139648,-2.3934197425842285,1.754178524017334,-4.45362663269043,-3.1286211013793945,-3.29793381690979,-9.266140937805176,2.5352697372436523,-1.3204454183578491,4.488137722015381,-13.05758571624756,2.132214069366455,-2.1768674850463867,-1.25927734375,1.398425817489624,3.5738015174865723,3.5107131004333496,-0.992900550365448,1.1035693883895874,1.7741847038269043,3.388300895690918,-3.3725361824035645,-4.215725898742676,-3.4468204975128174,-18.381467819213867,-1.2590954303741455,4.252473831176758,-4.764485836029053,0.7686983942985535,2.7725484371185303,0.9674292802810668,-3.902660369873047,-1.703576683998108,-0.0989984273910522,-1.7027876377105713,-0.3593013882637024,1.2180702686309814,0.7218268513679504,-0.6300607919692993,-12.700563430786133,-0.0013471841812133,5.832468509674072,1.5581425428390503,-0.9817917346954346,-3.70105242729187,0.8005427718162537,1.5881986618041992,-0.1362964510917663,3.4493885040283203,1.8612284660339355,-3.794499397277832,-5.5213165283203125,-6.081906795501709,-6.035520553588867,-14.458881378173828,-1.4955214262008667,-3.090846300125122,-2.392041683197021,-1.0948030948638916,7.123380184173584,2.59818959236145,0.6557443141937256,-6.606822967529297,0.3464055061340332,0.8755185008049011,6.181571006774902,2.0390281677246094,0.640335202217102,-0.5450219511985779,-0.268321543931961,2.2504591941833496,0.7428192496299744,2.423673629760742,-2.1642794609069824,-0.0660341382026672,0.1993255317211151,1.894345760345459,19.291004180908203,-1.221187710762024,2.266624212265014,-1.5937111377716064,-6.118683338165283,-4.387256145477295,2.151519775390625,-4.456537246704102,1.0658509731292725,-3.35552716255188,-1.0344618558883667,-19.605690002441406,1.194879174232483,-0.9556714296340942,2.093416690826416,3.520436286926269,-3.591493606567383,1.5110671520233154,4.603736877441406,-2.085824012756348,-2.329051733016968,3.742237091064453,4.138937950134277,-2.468945980072021,-0.6619378328323364,3.03313422203064,-0.708266019821167,3.2701988220214844,-3.638438224792481,-4.361441612243652,-7.439436912536621,2.834956169128418,9.386711120605469,-1.4885635375976562,3.701905250549317,-3.3051958084106445,-0.034755915403366,-1.8305010795593264,-4.046773910522461,-4.182580947875977,2.348219633102417,-1.453904628753662,0.2838583588600158,-0.6291296482086182,-0.916810154914856,-1.6025125980377195,-11.40635871887207,-1.7278151512145996,-3.760474443435669,8.39437484741211,1.7286498546600342,1.3523976802825928,-0.2520705461502075,1.5690133571624756,-14.487297058105469,2.5256404876708984,-22.05904769897461,5.4223408699035645,-2.962636232376098,-0.878184974193573,-0.4158579111099243,8.651719093322754,-1.000218629837036,-2.4287476539611816,4.939035415649414,3.321033000946045,-3.2407777309417725,-4.319506645202637,9.718995094299316,2.818056583404541,-3.030889749526977,1.5852768421173096,0.1058105379343032,2.578437805175781,-0.5969172120094299,1.7250995635986328,-1.7689521312713623,2.8603904247283936,-0.0331126004457473,0.890036940574646,-3.600393056869507,-3.0927939414978027,-0.0990954786539077,-11.264558792114258,-3.933107376098633,-3.0366458892822266,3.899700880050659,-1.1453676223754885,-3.3927814960479736,-5.540699005126953,3.898252487182617,-0.0175211429595947,3.094379425048828,5.111053466796875,3.233628749847412,7.425180435180664,-1.649770736694336,3.0641016960144043,5.074616432189941,2.0299246311187744,-4.957854747772217,0.6500045657157898,9.115571975708008,1.586619853973389,0.4814701080322265,-0.0277235507965087,1.943739414215088,2.040553569793701,-2.556711435317993,-1.5003215074539185,1.8115267753601076,1.0521067380905151,-3.240619659423828,2.525887012481689,-1.2790738344192505,-0.311370849609375,-3.0225789546966557,1.021713137626648,-0.2830151915550232,1.2206525802612305,-4.6806511878967285,0.2204005122184753,1.5413179397583008,4.159816265106201,-5.389308452606201,1.5925967693328855,-0.1456679105758667,-3.741607904434204,2.562716484069824,0.4974327087402344,-5.217159748077393,-5.7449798583984375,-2.2987401485443115,2.677142143249512,3.337979555130005,-2.221930503845215,1.921035885810852,3.4553799629211426,-1.0741063356399536,2.444852352142334,0.983063817024231,1.4914618730545044,3.0450246334075928,0.2097544074058532,2.134054660797119,-16.481887817382812,-1.5367276668548584,-3.1344943046569824,-3.4168131351470947,1.182725191116333,-0.6276742219924927,3.991343259811402,0.1498201787471771,-0.7142866253852844,-1.917647242546081,-1.444261908531189,-3.504415273666382,-1.831202507019043,-0.9409852623939514,-0.3887272179126739,-4.709577560424805,-2.40161681175232,1.374744534492493,-3.2108635902404785,3.058173656463623,-2.9241786003112797,-2.486469268798828,-4.881819725036621,-0.5841299891471863'

In [ ]:
speakvector_female = '-4.741,0.419,-3.355,3.652,-1.682,-1.254,9.719,1.436,0.871,12.334,-0.175,-2.653,-3.132,0.525,1.573,-0.351,0.030,-3.154,0.935,-0.111,-6.306,-1.840,-0.818,9.773,-1.842,-3.433,-6.200,-4.311,1.162,1.023,11.552,2.769,-2.408,-1.494,-1.143,12.412,0.832,-1.203,5.425,-1.481,0.737,-1.487,6.381,5.821,0.599,6.186,5.379,-2.141,0.697,5.005,-4.944,0.840,-4.974,0.531,-0.679,2.237,4.360,0.438,2.029,1.647,-2.247,-1.716,6.338,1.922,0.731,-2.077,0.707,4.959,-1.969,5.641,2.392,-0.953,0.574,1.061,-9.335,0.658,-0.466,4.813,1.383,-0.907,5.417,-7.383,-3.272,-1.727,2.056,1.996,2.313,-0.492,3.373,0.844,-8.175,-0.558,0.735,-0.921,8.387,-7.800,0.775,1.629,-6.029,0.709,-2.767,-0.534,2.035,2.396,2.278,2.584,3.040,-6.845,7.649,-2.812,-1.958,8.794,2.551,3.977,0.076,-2.073,-4.160,0.806,3.798,-1.968,-4.690,5.702,-4.376,-2.396,1.368,-0.707,4.930,6.926,1.655,4.423,-1.482,-3.670,2.988,-3.296,0.767,3.306,1.623,-3.604,-2.182,-1.480,-2.661,-1.515,-2.546,3.455,-3.500,-3.163,-1.376,-12.772,1.931,4.422,6.434,-0.386,-0.704,-2.720,2.177,-0.666,12.417,4.228,0.823,-1.740,1.285,-2.173,-4.285,-6.220,2.479,3.135,-2.790,1.395,0.946,-0.052,9.148,-2.802,-5.604,-1.884,1.796,-0.391,-1.499,0.661,-2.691,0.680,0.848,3.765,0.092,7.978,3.023,2.450,-15.073,5.077,3.269,2.715,-0.862,2.187,13.048,-7.028,-1.602,-6.784,-3.143,-1.703,1.001,-2.883,0.818,-4.012,4.455,-1.545,-14.483,-1.008,-3.995,2.366,3.961,1.254,-0.458,-1.175,2.027,1.830,2.682,0.131,-1.839,-28.123,-1.482,16.475,2.328,-13.377,-0.980,9.557,0.870,-3.266,-3.214,3.577,2.059,1.676,-0.621,-6.370,-2.842,0.054,-0.059,-3.179,3.182,3.411,4.419,-1.688,-0.663,-5.189,-5.542,-1.146,2.676,2.224,-5.519,6.069,24.349,2.509,4.799,0.024,-2.849,-1.192,-16.989,1.845,6.337,-1.936,-0.585,1.691,-3.564,0.931,0.223,4.314,-2.609,0.544,-1.931,3.604,1.248,-0.852,2.991,-1.499,-3.836,1.774,-0.744,0.824,7.597,-1.538,-0.009,0.494,-2.253,-1.293,-0.475,-3.816,8.165,0.285,-3.348,3.599,-4.959,-1.498,-1.492,-0.867,0.421,-2.191,-1.627,6.027,3.667,-21.459,2.594,-2.997,5.076,0.197,-3.305,3.998,1.642,-6.221,3.177,-3.344,5.457,0.671,-2.765,-0.447,1.080,2.504,1.809,1.144,2.752,0.081,-3.700,0.215,-2.199,3.647,1.977,1.326,3.086,34.789,-1.017,-14.257,-3.121,-0.568,-0.316,11.455,0.625,-6.517,-0.244,-8.490,9.220,0.068,-2.253,-1.485,3.372,2.002,-3.357,3.394,1.879,16.467,-2.271,1.377,-0.611,-5.875,1.004,12.487,2.204,0.115,-4.908,-6.992,-1.821,0.211,0.540,1.239,-2.488,-0.411,2.132,2.130,0.984,-10.669,-7.456,0.624,-0.357,7.948,2.150,-2.052,3.772,-4.367,-11.910,-2.094,3.987,-1.565,0.618,1.152,1.308,-0.807,1.212,-4.476,0.024,-6.449,-0.236,5.085,1.265,-0.586,-2.313,3.642,-0.766,3.626,6.524,-1.686,-2.524,-0.985,-6.501,-2.558,0.487,-0.662,-1.734,0.275,-9.230,-3.785,3.031,1.264,15.340,2.094,1.997,0.408,9.130,0.578,-2.239,-1.493,11.034,2.201,6.757,3.432,-4.133,-3.668,2.099,-6.798,-0.102,2.348,6.910,17.910,-0.779,4.389,1.432,-0.649,5.115,-1.064,3.580,4.129,-4.289,-2.387,-0.327,-1.975,-0.892,5.327,-3.908,3.639,-8.247,-1.876,-10.866,2.139,-3.932,-0.031,-1.444,0.567,-5.543,-2.906,1.399,-0.107,-3.044,-4.660,-1.235,-1.011,9.577,2.294,6.615,-1.279,-2.159,-3.050,-6.493,-7.282,-8.546,5.393,2.050,10.068,3.494,8.810,2.820,3.063,0.603,1.965,2.896,-3.049,7.106,-0.224,-1.016,2.531,-0.902,1.436,-1.843,1.129,6.746,-2.184,0.801,-0.965,-7.555,-18.409,6.176,-3.706,2.261,4.158,-0.928,2.164,-3.248,-4.892,-0.008,-0.521,7.931,-10.693,4.320,-0.841,4.446,-1.591,-0.702,4.075,3.323,-3.406,-1.198,-5.518,-0.036,-2.247,-2.638,2.160,-9.644,-3.858,2.402,-2.640,1.683,-0.961,-3.076,0.226,5.106,0.712,0.669,2.539,-4.340,-0.892,0.732,0.775,-2.757,4.365,-2.368,5.368,0.342,-0.655,0.240,0.775,3.686,-4.008,16.296,4.973,1.851,4.747,0.652,-2.117,6.470,2.189,-8.467,3.236,3.745,-1.332,3.583,-2.504,5.596,-2.440,0.995,-2.267,-3.322,3.490,1.156,1.716,0.669,-3.640,-1.709,5.055,6.265,-3.963,2.863,14.129,5.180,-3.590,0.393,0.234,-3.978,6.946,-0.521,1.925,-1.497,-0.283,0.895,-3.969,5.338,-1.808,-3.578,2.699,2.728,-0.895,-2.175,-2.717,2.574,4.571,1.131,2.187,3.620,-0.388,-3.685,0.979,2.731,-2.164,1.628,-1.006,-7.766,-11.033,-10.985,-2.413,-1.967,0.790,0.826,-1.623,-1.783,3.021,1.598,-0.931,-0.605,-1.684,1.408,-2.771,-2.354,5.564,-2.296,-4.774,-2.830,-5.149,2.731,-3.314,-1.002,3.522,3.235,-1.598,1.923,-2.755,-3.900,-3.519,-1.673,-2.049,-10.404,6.773,1.071,0.247,1.120,-0.794,2.187,-0.189,-5.591,4.361,1.772,1.067,1.895,-5.649,0.946,-2.834,-0.082,3.295,-7.659,-0.128,2.077,-1.638,0.301,-0.974,4.331,11.711,4.199,1.545,-3.236,-4.404,-1.333,0.623,1.414,-0.240,-0.816,-0.808,-1.382,0.632,-5.238,0.120,10.634,-2.026,1.702,-0.469,1.252,1.173,3.015,-8.798,1.633,-5.323,2.149,-6.481,11.635,3.072,5.642,5.252,4.702,-3.523,-0.594,4.150,1.392,0.554,-4.377,3.646,-0.884,1.468,0.779,2.372,-0.101,-5.702,0.539,-0.440,5.149,-0.011,-1.899,-1.349,-0.355,0.076,-0.100,-0.004,5.346,6.276,0.966,-3.138,-2.633,-3.124,3.606,-3.793,-3.332,2.359,-0.739,-3.301,-2.775,-0.491,3.283,-1.394,-1.883,1.203,1.097,2.233,2.170,-2.980,-15.800,-6.791,-0.175,-4.600,-3.840,-4.179,6.568,5.935,-0.431,4.623,4.601,-1.726,0.410,2.591,4.016,8.169,1.763,-3.058,-1.340,6.276,4.682,-0.089,1.301,-4.817'

In [ ]:
speakvector_female703 = '-3.28,3.23,1.11,1.59,2.57,-1.63,-1.84,-3.64,2.61,13.61,-0.78,1.93,-0.69,0.51,-2.56,4.26,-3.93,-3.81,2.99,-3.83,2.46,-5.33,-1.06,1.66,0.35,-3.06,-11.38,-12.25,-0.96,4.00,6.28,0.44,-0.66,-1.76,1.94,-1.31,3.37,-4.26,2.11,-1.56,3.26,0.34,3.23,3.75,-17.47,3.12,-4.34,-3.33,-3.57,-2.10,-2.27,8.02,-3.40,-0.93,6.49,-4.77,1.77,2.30,-0.70,1.89,-3.89,-1.21,3.64,-0.18,-0.34,2.26,2.36,7.65,-1.56,-4.49,-0.19,-0.05,-1.56,2.52,9.38,-2.04,-0.61,-9.90,-3.69,1.27,-0.74,-2.07,-1.69,9.01,7.34,-1.97,7.98,2.01,0.73,-1.27,-3.97,3.99,0.31,0.56,-2.93,-6.64,-3.84,-3.79,-1.25,-12.26,-1.30,-0.62,0.26,-0.44,-2.46,-0.37,1.63,-1.98,8.59,-1.25,-0.57,1.41,19.86,-0.45,-2.72,1.06,1.31,-0.52,-4.27,-1.91,-1.07,0.23,1.36,3.89,-1.97,3.16,-2.59,0.66,-2.54,-0.42,1.03,2.67,-2.54,1.02,-3.34,-3.14,-9.04,-0.63,8.84,-0.10,2.82,-0.28,0.44,-1.13,-1.22,-2.04,-0.38,-2.40,1.07,-1.20,0.15,0.01,1.44,17.43,1.22,-1.96,-13.67,-4.75,0.50,8.38,-2.09,-0.93,-2.08,-0.93,-0.44,-1.97,2.28,-1.11,1.45,-4.56,3.53,2.85,4.09,0.37,2.77,-0.26,0.22,0.51,-2.49,1.68,-2.12,-0.73,1.96,1.82,-2.22,2.65,-10.77,1.80,0.72,2.02,1.16,0.63,-3.97,-1.35,-2.14,-5.58,1.69,0.34,-4.67,1.52,-2.11,4.23,-0.30,1.99,-6.81,-1.51,4.23,7.29,-11.25,2.45,-1.73,-1.47,-0.81,-1.81,5.03,-1.85,-0.69,2.54,-6.28,15.93,-2.38,-6.94,0.16,7.19,4.10,-6.61,1.49,-0.89,0.03,-1.25,0.88,7.14,-0.07,-4.89,-2.58,-1.84,0.23,0.81,0.66,-1.61,-8.00,-1.11,3.14,-0.43,-0.46,-1.19,3.31,1.03,1.82,0.74,-3.03,-2.43,1.31,-1.25,12.32,-1.95,6.51,-4.65,-1.37,-3.51,0.90,-0.82,1.82,-0.70,-0.76,-3.24,7.56,4.88,4.72,0.33,3.82,3.28,-2.73,2.21,-2.52,-8.72,-7.55,-1.71,2.36,2.21,2.66,-2.51,-0.62,-2.30,1.19,2.92,1.02,-3.18,1.88,-2.63,1.19,1.29,2.89,-1.49,-4.38,-5.68,3.38,-16.83,-3.35,-2.27,-1.00,2.81,-3.38,-2.36,0.27,-4.84,13.44,-0.93,0.03,1.94,-4.00,-0.60,10.54,-3.31,1.84,-2.82,3.45,6.83,1.31,1.63,1.95,5.47,-1.48,1.24,-2.22,2.98,2.13,2.88,-1.91,1.74,-0.35,11.95,-2.06,6.44,0.06,-1.46,-18.77,1.49,-4.99,0.21,0.42,3.51,6.92,-0.07,-0.22,-0.24,-0.32,-0.29,2.45,13.04,-6.34,5.00,-1.99,1.25,-0.40,-1.48,1.84,0.63,2.20,-4.34,2.92,-0.46,9.56,2.94,3.58,-8.99,-0.63,-0.29,3.57,7.26,-0.72,-1.74,0.77,2.89,9.85,-1.73,0.23,-2.79,-11.43,-2.19,-10.12,2.39,0.73,-1.38,0.29,1.41,4.13,2.43,6.93,-0.20,-0.44,3.19,0.14,0.67,-3.70,-1.29,-0.15,-1.92,-3.23,-1.93,3.08,2.79,4.08,3.37,10.38,-2.45,-1.49,1.39,3.11,-16.65,-0.22,0.44,3.55,-1.99,-2.69,-4.42,9.84,3.85,-0.55,3.32,-1.86,4.12,1.53,-2.92,-2.71,4.32,1.72,16.17,0.87,5.54,4.91,-1.92,-2.20,-1.70,1.66,-13.42,1.77,0.92,-0.03,1.24,-0.65,0.83,-1.39,-0.54,3.47,-2.99,8.87,-0.33,-4.29,-0.45,0.01,-1.30,4.03,-1.95,0.90,4.42,-1.30,3.53,-0.39,-6.40,-5.69,-0.98,-1.15,-6.21,3.11,-2.35,7.45,9.36,-0.54,3.64,1.34,-4.40,-0.09,4.75,-5.70,1.45,-1.34,4.57,-1.12,1.90,0.71,9.11,3.59,0.75,-2.24,-2.21,-2.31,-1.20,-1.66,-2.11,2.72,2.20,6.06,3.54,-3.63,-3.90,0.92,2.79,-5.76,0.68,-0.95,0.77,2.52,5.81,-4.46,-4.06,0.78,2.83,5.42,0.32,-0.26,-1.15,-8.20,1.10,-3.08,-6.55,-3.63,3.16,2.66,-3.30,-2.02,2.46,0.07,3.78,-2.78,1.71,-3.46,0.50,-10.14,-2.96,0.00,6.55,-3.28,2.22,0.52,-0.33,9.22,9.06,4.56,6.37,-1.81,-8.68,-2.49,-0.52,-14.50,-0.91,-4.35,-1.01,5.37,3.34,2.87,0.85,-1.90,-0.79,1.27,-7.01,-3.85,1.90,8.97,0.46,-0.06,-2.59,1.50,-1.81,-1.24,-0.71,-1.00,-1.96,-2.06,0.25,-1.61,-3.75,-1.96,1.91,-5.38,-3.17,-14.13,-0.99,0.77,-0.10,1.12,-5.05,-1.83,0.51,-0.91,4.91,-0.48,-1.14,-7.53,-1.48,0.21,-1.20,3.80,1.25,2.24,-0.88,3.13,2.63,-0.50,-5.09,2.70,1.35,1.37,1.94,2.19,-1.88,2.46,-2.39,0.36,-6.38,-10.66,-0.26,-2.02,0.52,-0.02,-4.61,1.08,-2.40,-1.36,-5.14,-0.19,-3.95,2.90,-4.20,0.74,-3.72,-3.47,3.77,7.43,-1.59,-0.99,0.43,0.43,2.62,-10.30,-0.30,3.25,2.95,-6.60,-0.58,3.39,-1.54,-5.16,6.08,4.03,-13.07,4.30,2.62,9.62,4.26,7.05,0.53,-1.00,-9.13,3.72,-10.53,-3.92,-8.09,2.11,-0.28,-6.41,-1.40,-3.28,1.16,-0.59,-6.07,6.34,-10.22,8.61,-2.03,7.20,-6.97,-0.41,-2.52,0.80,0.13,-1.66,2.40,0.63,-10.22,-6.31,3.25,-0.41,-0.24,-3.47,-1.28,7.12,-1.37,0.86,1.14,3.57,-1.62,-0.08,1.49,-21.37,0.41,-2.50,-13.28,-0.28,-7.22,0.91,3.73,-0.24,-0.66,-2.86,-1.12,-7.34,-2.23,1.28,6.27,1.85,-0.62,-0.16,2.32,-1.92,1.27,0.77,4.18,1.55,0.04,-7.49,1.15,2.03,-0.99,0.53,-1.84,-2.96,0.25,-0.38,1.01,-4.31,4.57,-0.33,2.58,0.26,4.11,0.20,1.57,9.60,2.80,4.44,3.85,2.20,3.94,-2.01,1.88,-2.29,0.27,-1.82,-0.33,2.28,-4.80,-0.95,0.92,5.23,2.21,0.31,1.11,4.68,0.69,-1.36,1.30,-1.82,-1.73,-3.87,1.95,-0.47,-5.13'

In [34]:
speakvector_female545 = '3.23,3.16,13.50,5.19,5.36,-6.72,12.94,-0.48,0.14,0.42,6.63,-1.55,-1.50,0.79,0.37,1.79,-4.59,2.55,-0.47,6.55,-6.65,-0.05,1.18,9.82,3.78,-0.63,-9.94,-11.90,-0.35,0.91,-3.57,-0.47,2.10,-0.13,-2.04,0.69,2.63,2.26,-2.45,2.32,1.84,-0.05,3.72,4.91,-11.71,3.02,0.54,13.55,1.59,4.16,1.50,2.12,4.82,1.87,1.68,-1.28,-1.39,-0.59,1.35,0.33,2.63,-1.47,1.31,0.29,1.09,-0.22,2.37,16.84,2.30,3.34,-4.35,0.78,-4.58,2.60,7.09,-1.37,1.60,-7.34,2.62,1.08,-3.00,-9.60,2.51,1.95,2.81,0.63,0.41,-1.72,0.17,-4.51,1.62,1.03,3.49,0.95,4.85,-7.04,-2.75,-0.02,-2.41,-8.28,0.34,1.43,-0.51,1.03,-4.22,-2.06,2.88,-1.68,15.65,0.45,1.25,13.34,3.84,0.65,-0.54,2.17,-1.50,2.99,-6.15,1.49,-2.45,-0.34,-5.55,-0.34,3.14,-1.57,-3.61,0.59,1.86,1.03,2.19,-1.31,0.44,-0.12,17.54,1.02,-3.01,-3.34,-7.94,-3.22,-0.12,-0.73,-1.91,4.52,0.28,0.04,1.28,-7.36,0.29,1.62,4.73,-3.77,-1.64,2.15,-2.74,-2.74,-7.59,2.52,-1.74,-7.34,1.90,-1.36,-0.76,-4.95,1.65,4.19,1.94,-5.02,4.51,-4.47,0.26,-0.33,-3.39,-0.63,0.46,0.01,3.23,1.22,-0.54,-0.27,-0.56,5.62,-3.36,-2.81,-0.93,2.20,-18.49,1.18,0.52,0.86,-1.50,-3.18,4.00,1.55,-3.87,-10.39,3.05,3.43,-2.34,1.50,-4.34,-3.01,-1.67,5.72,6.45,1.37,1.22,5.04,2.63,-3.89,0.04,-4.92,0.37,-0.24,3.57,2.64,1.40,1.78,6.11,0.34,0.69,-2.68,1.46,4.50,1.37,1.48,-1.91,1.96,-1.28,4.01,0.78,-6.64,2.00,2.78,-3.93,4.38,3.41,-0.11,-4.35,-1.88,1.31,4.70,-6.64,-5.77,1.01,1.91,-4.31,-7.53,3.07,3.45,3.74,2.82,-2.95,-0.04,-14.35,5.68,5.15,8.03,1.71,1.18,1.39,-4.41,-2.63,-3.14,2.87,-0.39,-1.97,-2.99,3.68,2.40,-1.31,5.06,-5.02,0.32,2.58,-5.43,4.27,0.04,-4.81,0.84,2.54,-0.04,-1.48,0.64,-0.25,-2.49,-4.18,2.56,-0.87,-0.03,-2.79,5.95,2.18,-2.26,1.40,1.18,-1.42,-5.92,0.23,1.33,1.88,1.88,2.46,-0.20,1.86,-0.31,11.37,0.46,-2.85,-0.70,-2.47,0.89,3.95,3.99,-2.26,-0.36,0.22,6.93,1.32,0.29,-4.19,-4.01,-0.81,-1.75,1.19,-3.24,-0.48,15.49,-2.34,6.41,2.74,-3.54,-4.25,-0.29,-4.71,2.96,4.33,16.26,-5.26,3.18,2.08,-3.13,-3.93,0.94,-1.61,-1.01,0.78,-2.19,-1.90,8.74,1.40,2.71,4.36,-3.46,-0.91,-0.66,-3.57,1.12,-3.08,0.91,-3.49,1.23,7.70,-1.24,4.08,-10.69,4.16,-1.97,3.08,1.48,2.40,-2.53,1.55,4.04,12.14,-5.26,-3.83,1.82,0.02,-1.99,5.20,-1.36,2.24,-15.76,-2.14,6.45,-0.83,0.48,2.99,-0.33,4.45,4.51,-7.12,-3.34,6.34,-7.61,2.98,-0.77,2.81,-1.72,0.70,2.33,-1.20,-1.46,1.12,-2.17,4.05,1.20,4.11,-14.63,-1.91,-0.45,-2.87,-0.11,-0.85,1.52,12.19,-0.65,5.04,5.59,1.27,3.87,-0.01,-3.02,0.67,3.07,6.73,9.47,0.05,0.99,3.82,-1.03,-1.23,-1.33,-2.70,6.56,-4.93,-3.02,2.64,-1.65,2.14,-1.81,-1.72,-5.94,1.26,-1.45,-8.16,1.83,-1.06,1.47,0.04,3.21,-0.12,-0.84,-0.26,0.64,-2.91,4.61,-8.54,-2.23,2.03,-1.13,0.49,-4.29,-2.09,-0.45,4.66,-1.25,4.66,2.27,3.19,3.47,3.08,-0.54,1.90,-1.85,-2.94,11.56,3.53,2.14,-3.18,0.66,2.56,-0.71,-0.05,4.06,0.26,0.37,9.71,5.19,-1.60,-6.96,-3.11,5.49,6.04,2.59,-0.87,-3.76,-2.40,0.62,1.36,-3.84,5.42,-5.75,7.10,-10.53,3.40,-5.79,2.34,-0.49,-0.53,0.36,-4.25,0.02,3.66,6.00,-0.09,-2.17,-0.88,-5.30,-0.78,2.96,-5.71,-4.48,-1.19,0.45,-14.38,-0.54,0.95,-0.04,-2.56,1.94,-8.78,-2.43,0.25,0.09,-5.43,-21.73,-1.57,-2.53,3.67,-11.51,4.68,-3.14,-20.38,4.74,-7.40,-0.97,4.06,-1.13,0.82,1.01,3.84,2.92,-5.43,-7.24,-3.44,-2.58,4.69,-8.31,3.44,-5.82,-2.43,-2.24,-0.30,-18.85,-6.89,-0.13,5.18,3.74,5.36,4.01,3.00,-2.78,-3.88,16.17,11.96,8.42,-1.02,-1.96,-0.75,-8.65,-1.14,-2.00,-1.83,4.23,0.89,0.59,0.33,-5.58,3.46,0.15,-0.03,-1.88,-2.03,-6.96,-0.98,3.07,3.04,15.59,1.29,-2.19,0.97,3.06,1.84,2.27,-1.07,-3.59,0.53,-3.73,-14.13,0.99,-0.87,2.43,5.22,-0.02,0.95,2.74,-2.90,2.25,-4.94,-4.73,-0.28,3.01,2.18,-1.24,-0.23,1.92,5.50,4.04,-0.20,-7.52,0.08,2.03,3.20,4.94,-3.41,-2.56,-11.01,1.10,0.90,-0.55,-1.61,-1.93,-2.39,-2.76,0.25,-4.26,3.28,-4.51,-0.06,-5.04,-3.33,-4.27,3.64,-16.57,1.57,9.29,-4.84,4.96,-0.89,-5.67,-4.57,3.84,-3.27,-0.81,1.09,10.73,-5.31,-1.75,0.12,4.19,-0.55,-1.50,1.27,-2.14,4.66,-0.13,-10.97,-2.45,0.12,0.67,-6.79,5.15,5.37,-0.75,3.75,0.15,6.55,-1.66,-1.33,3.10,-9.60,2.82,2.55,-6.66,1.64,6.33,0.29,-17.29,1.60,-0.02,-0.21,-1.50,-2.46,-0.93,1.02,1.15,2.24,1.26,-0.13,-2.62,3.17,-1.81,-3.53,0.45,-0.98,1.06,-4.55,3.41,-5.00,0.21,7.20,-0.82,-0.73,4.23,0.46,4.03,0.97,-0.27,5.95,1.69,0.26,-2.29,-8.59,-3.39,2.97,1.82,0.75,-0.40,1.52,5.53,-1.15,5.72,-8.11,-0.62,0.80,4.73,3.00,-3.58,-5.26,-1.98,-0.11,-0.62,1.10,-2.33,1.44,-3.04,3.03,4.26,1.24,-2.45,3.51,1.24,1.14,1.02,-0.98,-3.08'

In [38]:
speakvector_female523 = '1.86,5.08,8.13,1.16,0.52,2.51,-25.07,-1.74,-5.08,10.00,2.44,-4.08,2.00,3.27,-3.37,-0.98,0.18,1.77,-1.50,-5.24,-6.47,2.31,2.66,25.55,-1.63,-0.83,10.65,-8.74,-5.67,-4.64,2.43,-2.83,-3.77,4.16,-2.65,-15.29,-5.75,0.89,-3.28,2.75,1.47,4.67,2.69,-0.66,-9.10,1.05,0.85,-6.02,4.32,-0.93,-2.99,-0.91,0.60,-0.81,-1.05,3.56,0.86,-0.70,-2.23,-0.19,0.05,-1.88,2.29,-0.09,-4.87,-4.26,0.62,-10.22,3.39,-1.74,2.26,-2.54,-2.35,1.29,-5.27,-1.29,-5.09,-1.98,-0.29,-4.80,-2.46,-4.89,2.26,-0.03,0.32,-2.09,7.58,-1.07,-3.16,-2.93,19.93,-0.86,5.33,-0.26,4.95,-9.72,0.10,1.36,-6.33,-2.14,-2.56,-0.21,2.73,-1.17,4.56,-1.87,-0.64,-2.68,5.84,-5.03,-0.49,12.88,1.19,-0.30,0.88,1.06,0.10,-0.31,4.63,-0.36,-1.39,1.73,-2.00,3.17,11.32,1.07,-6.54,-1.61,5.69,1.45,0.40,-0.16,0.63,-0.54,-0.91,-0.32,4.18,0.88,-2.23,-4.76,2.58,0.78,-1.83,-2.84,1.84,4.06,-2.93,-1.31,1.01,-0.32,5.80,1.53,3.54,5.12,2.49,-3.47,-6.71,0.69,-3.16,-0.92,5.77,0.12,-1.17,7.36,1.68,0.94,0.65,-2.30,-1.31,-3.83,6.91,-5.75,-1.22,-1.83,-0.31,2.64,2.17,2.62,-0.26,0.18,5.01,2.15,2.55,5.14,1.74,-0.14,-16.24,0.90,0.96,0.29,-0.91,-3.04,0.83,-3.14,1.48,-5.59,0.70,4.65,-2.03,-3.99,2.34,-4.75,-4.87,0.65,1.05,-1.57,0.95,11.93,8.09,-0.24,4.02,-4.24,-0.10,-3.55,-3.22,-0.19,-1.98,-5.58,-0.05,-5.62,-2.23,-2.43,-0.13,-2.31,3.46,-4.86,-4.95,3.57,0.15,1.19,3.34,2.02,0.58,-0.61,2.22,7.16,2.10,4.06,3.68,-6.81,4.58,2.67,-9.84,-2.40,-2.24,0.78,-5.41,2.21,25.15,-1.49,-3.57,1.99,0.19,2.08,12.48,0.01,11.01,-4.71,0.30,4.67,2.96,-3.16,3.62,1.82,-0.64,2.03,0.53,4.02,1.56,-0.11,-0.62,1.67,-9.44,1.94,-1.73,0.31,-4.05,-0.45,3.50,1.40,-0.92,-0.46,-2.03,0.48,-3.15,3.04,-3.99,0.58,-0.81,-3.18,1.66,-1.94,-1.28,-1.43,-4.03,-3.57,0.37,-15.98,-1.59,-1.75,5.07,-1.41,-2.51,1.72,1.69,-0.73,3.31,0.25,1.05,-0.83,-4.42,5.64,1.98,6.44,2.13,3.37,6.67,4.32,-0.90,5.94,1.91,3.40,2.10,0.32,-3.21,4.46,5.05,3.27,-0.60,-1.76,-0.12,-0.52,-2.59,-11.65,2.41,5.89,-7.50,14.76,-3.37,-0.13,4.31,0.19,-1.16,0.08,2.04,3.63,0.85,-1.73,0.01,2.29,0.83,2.80,2.40,0.77,0.01,-6.03,2.82,-2.06,-1.47,1.60,-3.19,-8.37,-5.27,-2.85,-2.11,-12.71,5.19,-7.19,-2.22,-2.46,-2.29,2.27,-4.95,-4.86,7.72,0.74,-4.38,1.55,-0.53,-2.85,-1.38,-0.89,0.56,-1.21,1.37,-2.65,-3.12,-4.30,-4.09,0.42,2.15,-7.70,-3.07,-6.96,-6.76,-0.19,4.88,-4.84,-3.12,-4.35,4.04,0.16,-3.90,-0.29,-5.84,0.38,-1.74,0.98,0.67,-5.41,-3.09,-0.06,-3.37,-0.74,-2.48,-5.36,5.05,-3.89,2.38,5.88,1.29,0.16,-3.13,-6.34,-3.08,0.80,2.31,15.87,-0.61,4.36,-1.83,-3.95,3.22,-5.86,-4.09,3.33,-2.55,4.84,-4.07,-2.50,-0.46,0.13,4.35,1.95,-4.13,-2.12,1.60,4.68,-0.33,3.63,1.95,0.07,-7.12,1.02,-0.56,-3.86,-2.70,3.57,-2.27,0.88,8.11,-5.13,1.77,-7.46,1.82,0.74,-0.25,-15.23,4.89,6.96,2.98,8.22,0.24,-5.12,-0.91,-3.44,3.22,10.54,0.93,-1.88,-0.16,2.27,1.31,3.48,-2.96,-1.06,1.24,-3.46,-5.54,4.39,-4.29,-0.73,8.82,0.74,-5.59,5.24,0.70,4.82,1.97,1.50,-0.18,-4.99,1.79,-2.13,-4.88,-7.93,4.55,-0.22,0.69,-2.02,1.31,0.97,1.68,-1.08,0.69,-8.93,-2.87,1.34,1.56,1.16,-5.79,1.45,-6.54,-0.08,0.00,1.93,-3.26,1.05,6.23,-1.33,-3.81,4.26,3.06,-2.92,-3.96,-1.00,3.62,9.87,-0.92,8.26,-1.61,-2.11,0.19,0.13,-18.09,-0.23,-10.28,-0.95,1.72,4.00,0.30,-1.82,3.32,3.29,-8.01,-1.53,5.52,9.26,2.62,2.29,-0.40,-0.45,5.40,5.07,1.84,-0.76,-3.12,2.42,-1.46,1.64,5.34,0.91,2.05,2.47,-0.06,-3.52,-10.03,0.31,-2.56,1.17,-1.28,-3.53,1.52,10.22,0.46,7.57,2.25,-3.52,2.66,1.40,-0.67,2.71,3.19,1.02,-1.66,-3.38,1.34,1.20,-0.35,12.59,1.60,2.00,1.24,-0.15,1.68,-2.34,-2.24,-6.69,-5.79,-2.64,-17.58,-0.17,2.15,3.90,-4.02,4.36,5.96,0.47,-0.68,-5.54,5.42,-2.29,-4.33,-0.55,3.26,-0.07,-0.14,-1.45,-2.23,-1.16,0.74,6.18,2.48,-3.84,-0.71,5.25,-1.48,2.31,-6.01,-0.01,-4.88,0.48,-7.87,-3.96,-2.66,0.10,0.51,-3.46,-6.83,1.02,1.98,2.15,0.31,-7.46,-2.29,0.33,3.13,-1.07,-2.47,0.35,-5.91,-1.49,-5.82,0.91,-0.89,0.52,1.79,9.07,13.55,-1.85,-1.71,-5.97,2.33,0.03,1.89,-0.75,-3.42,-1.33,-1.15,3.31,1.10,3.10,13.63,2.16,0.59,4.93,-7.43,3.15,2.32,0.64,7.42,-4.08,-2.66,1.06,-2.00,0.94,-1.96,-9.27,0.44,4.63,-2.21,5.70,-7.85,-1.99,-5.43,2.72,-0.59,1.91,0.79,2.83,3.83,1.30,-0.71,-3.54,0.56,-3.05,2.63,3.32,-1.73,-6.68,1.12,-0.88,-2.04,0.59,2.00,-1.05,2.25,2.65,0.43,-2.44,0.03,-0.55,-0.68,1.25,-0.16,-4.30,-2.16,0.26,8.26,-1.53,-0.23,-0.54,3.06,-0.43,-11.32,-9.21,-3.67,2.02,1.27,-0.85,4.64,2.10,-1.69,3.44,-0.29,0.71,-0.49,-2.37,-0.99,3.26,2.75,-0.75,-1.97,5.34,0.28,0.47,-1.16,-1.03'

In [48]:
speaker = torch.tensor([float(x) for x in speakvector_male707.split(',')])

In [52]:

params_infer_code = ChatTTS.Chat.InferCodeParams(
    prompt="[speed_2]",
    temperature=0.3,
    top_P = 0.9, 
    top_K = 1,
    spk_emb = speaker
)
params_refine_text = ChatTTS.Chat.RefineTextParams(
    prompt="[oral_0][laugh_0][break_6]",
)
# break 0-7
# oral 
# laugh 0-2

wav = chat.infer(
    "四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片等，每样都让人垂涎三尺。",
    params_refine_text=params_refine_text,
    params_infer_code=params_infer_code,
)

[+0800 20240909 17:34:25] [INFO] ChatTTS | norm | replace homophones: 涎->闲
text:  13%|█▎        | 51/384(max) [00:00, 82.14it/s]
code:  21%|██▏       | 439/2048(max) [00:05, 83.16it/s]


In [50]:
Audio(wav[0], rate=24_000, autoplay=True)

### fix random speaker

In [ ]:
rand_spk = chat.sample_random_speaker()
print(rand_spk)  # save it for later timbre recovery

params_infer_code = ChatTTS.Chat.InferCodeParams(
    spk_emb=rand_spk,
)

wav = chat.infer(
    "四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。",
    params_infer_code=params_infer_code,
)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Zero shot (simulate speaker)

In [ ]:
from ChatTTS.tools.audio import load_audio

spk_smp = chat.sample_audio_speaker(load_audio("sample.mp3", 24000))
print(spk_smp)  # save it in order to load the speaker without sample audio next time

params_infer_code = ChatTTS.Chat.InferCodeParams(
    spk_smp=spk_smp,
    txt_smp="与sample.mp3内容完全一致的文本转写。",
)

wav = chat.infer(
    "四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。",
    params_infer_code=params_infer_code,
)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Two stage control

In [ ]:
text = "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with."
refined_text = chat.infer(text, refine_text_only=True)
refined_text

In [ ]:
wav = chat.infer(refined_text, skip_refine_text=True)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

## LLM Call

In [ ]:
from ChatTTS.tools.llm import ChatOpenAI

API_KEY = ""
client = ChatOpenAI(
    api_key=API_KEY, base_url="https://api.deepseek.com", model="deepseek-chat"
)

In [ ]:
user_question = "四川有哪些好吃的美食呢?"

In [ ]:
text = client.call(user_question, prompt_version="deepseek")
text

In [ ]:
text = client.call(text, prompt_version="deepseek_TN")
text

In [ ]:
wav = chat.infer(text)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)